In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix
import os
from os import listdir
from os.path import isfile, join

In [ ]:
#--------ATTENTION: HERE CHANGE THE PATH DEPENDING ON YOUR DATA LOCATIONS -------
basePath = "/Users/macbookair/Desktop/università/computational intelligence/truth_predictor_project/models-data/data-from-eeglab/"
spectraBasePath = "/Users/macbookair/Desktop/università/computational intelligence/truth_predictor_project/spectra_epoch/"

# data frame of all the time domain target data
eegDataTarget = pd.DataFrame()
# data frame of all the time domain NOT target data
eegDataNotTarget = pd.DataFrame()

# data frame of all the spectra target data
spectraDataTarget = pd.DataFrame()
# data frame of all the spectra NOT target data
spectraDataNotTarget = pd.DataFrame()

allSubjectList = os.listdir(basePath)

# for all the subjects names contained in the directory "basePath"
for subjectName in allSubjectList:
    # if the subject is actually a directory
    if os.path.isdir(basePath + subjectName):
        # get the list of files names of a single subject directory
        singleSubjectFiles = os.listdir(basePath + subjectName)
        # for all files names in the subject directories
        
        for targetFileName in  singleSubjectFiles:
            # if the file is a target file contains the string "epIs" and also if it ends with EEG.csv (that is time domain data)
            if "epIs" in targetFileName and "EEG.csv" in targetFileName:
                # put file name in target array
                targetFile = pd.read_csv(basePath + subjectName + "/" + targetFileName,sep="\t")
                
                # concatenate the time domain TARGET data with the previously concatenated one coming from the other
                # time domain EEG files taken in previous iterations of this for loop
                eegDataTarget = pd.concat([eegDataTarget,targetFile])
                
                # with different path the NAME of a spectra file is equal to a "time domain EEG file", it changes
                # only the fact that istead of EEG.CSV the name ends with SPECTRA.CSV
                spectraFileNameTarget = spectraBasePath + targetFileName.replace("EEG.csv","SPECTRA.csv")
                
                # Get the spectra file
                if os.path.exists(spectraFileNameTarget):
                    spectraTargetFile = pd.read_csv(spectraFileNameTarget, header=None)
                else:
                    # in case the file does not exists there is a variant of SPECTRA FILE that ends with
                    # EEG-SPECTRA.csv
                    spectraTargetFile = pd.read_csv(spectraBasePath + targetFileName.replace("EEG.csv","EEG-SPECTRA.csv"), header=None)
                
                # concatenate the spectra TARGET data with the previously concatenated one coming from the other
                # spectra files taken in previous iterations of this for loop
                spectraDataTarget = pd.concat([spectraDataTarget,spectraTargetFile])
                
            elif "epNot" in targetFileName and "EEG.csv" in targetFileName:
                # the logic is the same of the above else, but is about NOT Targets data
                notTargetFile = pd.read_csv(basePath + subjectName + "/" + targetFileName,sep="\t")
                eegDataNotTarget = pd.concat([eegDataNotTarget,notTargetFile])
                spectraFileNameNotTarget = spectraBasePath + targetFileName.replace("EEG.csv","SPECTRA.csv")
                
                if os.path.exists(spectraFileNameNotTarget):
                    spectraNotTargetFile = pd.read_csv(spectraFileNameNotTarget, header=None)
                else:
                    spectraNotTargetFile = pd.read_csv(spectraBasePath + targetFileName.replace("EEG.csv","EEG-SPECTRA.csv"), header=None)
                    
                spectraDataNotTarget = pd.concat([spectraDataTarget,spectraNotTargetFile])

In [ ]:
# for a strange reason the time column has an empty name, so rename it to "Time" for target data
eegDataTarget.rename(index=str, columns={" ": "Time"},inplace=True)
#for another strange reason an empty column is in the TARGET dataframe with name "Unnamed: 5" so let's remove it
del eegDataTarget["Unnamed: 5"]

# same operations as above but for not targets
eegDataNotTarget.rename(index=str, columns={" ": "Time"},inplace=True)
del eegDataNotTarget["Unnamed: 5"]

In [ ]:
# window start represents when we do we cut the data starting from the card shown event
# e.g. 0-3000 means from the moment the card has been shown until 3 seconds after
# REMEMBER THE DATA WINDOW HAS BEEN TAKEN WITH -1000:3000ms THIS ARE THE LIMITS OF THE WINDOW START
# AND WINDOW END WE CAN HAVE
windowStart = 0
windowEnd = 3000

# number of seconds of the windows (e.g. 3 seconds in case of an interval 0-3000ms)
numberOfSeconds = (windowEnd - windowStart)/ 1000

# elaboration of the number of sampled data for each round, that is 256 data points per second
# in the e.g. 0-3000 it is: (3sec * 256sampling rate)
numberOfSamplesPerRound = int(numberOfSeconds * 256)

#cut the target and not target data with the decided window length
eegDataTargetReduced = eegDataTarget[(eegDataTarget["Time"] >= windowStart) & (eegDataTarget["Time"] < windowEnd)]
eegDataNotTargetReduced = eegDataNotTarget[(eegDataNotTarget["Time"] >= windowStart) & (eegDataNotTarget["Time"] < windowEnd)]

In [ ]:
# create an array which separates all the concatenated time domain data in different epochs, each epoch in the array
# is an array containing all the samples data representing the epoch (e.g. data in 0-3000ms).
# the creation of the array is useful to concatenate the EEG time domain data with the spectra, and for then 
# flatten it for the classifier by having this result as a row "concatenate(EEG_subject_time_domain,subject_spectra)""
eegDataTargetReducedNumpy = np.array(eegDataTargetReduced.values)
numberOfTargetValues = int(eegDataTargetReduced.shape[0]/numberOfSamplesPerRound)
trainingTargetSet = np.array(np.array_split(eegDataTargetReducedNumpy, eegDataTargetReduced.shape[0]/numberOfSamplesPerRound))

#do the same of above for not target
eegDataNotTargetReducedNumpy = np.array(eegDataNotTargetReduced.values)
trainingNotTargetSet = np.array(np.array_split(eegDataNotTargetReducedNumpy, eegDataNotTargetReduced.shape[0]/numberOfSamplesPerRound))


In [ ]:
# do the same of above that has been done in time domain, this time is done for SPECTRA type data that will be
# concatenated with the time domain
spectraDataTargetReducedNumpy = np.array(spectraDataTarget.values)
trainingSpectraTargetSet = np.array(np.array_split(spectraDataTargetReducedNumpy, spectraDataTarget.shape[0]/4))

spectraDataNotTargetReducedNumpy = np.array(spectraDataNotTarget.values)
trainingSpectraNotTargetSet = np.array(np.array_split(spectraDataNotTargetReducedNumpy, spectraDataNotTarget.shape[0]/4))
spectraDataTargetReducedNumpy.shape

In [ ]:
trainingTargetSetReshaped = []
trainingNotTargetSetReshaped = []
#create the arrays of flattened data for TARGET time domain and spectra
for i in range(numberOfTargetValues - 1):
    trainingTargetSetReshaped.append(np.hstack((trainingTargetSet[i].flatten(),trainingSpectraTargetSet[i].flatten())))
    
#create the arrays of flattened data for NOT TARGET time domain and spectra
for i in range(numberOfTargetValues - 1):
    trainingNotTargetSetReshaped.append(np.hstack((trainingNotTargetSet[i].flatten(),trainingSpectraNotTargetSet[i].flatten())))
    
trainingTargetSetReshaped = np.array(trainingTargetSetReshaped)
trainingNotTargetSetReshaped = np.array(trainingNotTargetSetReshaped)

print(trainingTargetSetReshaped.shape)
print(trainingNotTargetSetReshaped.shape)

In [ ]:
# 
trainingTargetSetReshapedAndAveraged = np.zeros((trainingTargetSetReshaped.shape[0],trainingTargetSetReshaped.shape[1]))
trainingNotTargetSetReshapedAndAveraged = np.zeros((trainingTargetSetReshaped.shape[0],trainingTargetSetReshaped.shape[1]))

# this array can be used to check more window sizes for the smoothing made with the average, 
# the current best found window size is around 15, so to see one single risult for a smoothing of 15 on each size
# set the variable to [15], otherwise you can try more window sizes in this way e.g.: [13,15,16,22,25]
arrayOfWindowSizes = [15]

for i in arrayOfWindowSizes:
    numberOfElectrodesLinesToBeAveragedPerSide = i
    
    print("Window size: ",end="")
    print(numberOfElectrodesLinesToBeAveragedPerSide * 2)
    
    numberOfElectrodesLinesToBeAveraged = (numberOfElectrodesLinesToBeAveragedPerSide * 2) + 1
    # the number of considered electrodes plus the time if in the data. The maximum value is 
    # 4 electrodes + time var = 5.
    # if, for instance, we removed 2 electrodes it will be 
    # 2 electrodes + time = 3
    # if we removed the time attribute from the data it will be 
    # 2 electrodes = 2
    numberOfAttributesConsidered = 5
    
    # a data row is a set of sampled data flattened, to do the average there must be some jumps. To understand this
    # consider the structure of two samples where each electrode is named E:
    # (E1.1, E2.1, E3.1, E4.1, T1, E1.2, E2.2, E3.2, E4.2, T2)
    # to make the average smoothing the index of E1.1 must be averaged with the index of E1.2 that is
    # E1.1.index + numberOfAttributesConsidered, this reasoning has to be repeated for all the samples in all
    # the time domain considered that was defined by startWindow and endWindow
    totalNumberOfAttributesToJump = numberOfElectrodesLinesToBeAveragedPerSide * numberOfAttributesConsidered

    # the data points considered are equal to the sampling rate * number of seconds in epoch * the number of attributes
    timeDomainNumberOfDataFlattened = 256 * numberOfSeconds * numberOfAttributesConsidered
    
    # START AVERAGE SMOOTHING
    # this code is relatively intricate and does the average smoothing for TARGET and NOT TARGEt
    for i in range(trainingTargetSetReshaped.shape[0]):
        for j in range(trainingTargetSetReshaped.shape[1]):
            if j < timeDomainNumberOfDataFlattened and j >= totalNumberOfAttributesToJump and j < trainingTargetSetReshaped.shape[1] - totalNumberOfAttributesToJump:
                z = 0
                while z <= numberOfElectrodesLinesToBeAveragedPerSide:
                    if z != 0:
                        trainingTargetSetReshapedAndAveraged[i][j] += trainingTargetSetReshaped[i][j - (numberOfAttributesConsidered * z)]

                    trainingTargetSetReshapedAndAveraged[i][j] += trainingTargetSetReshaped[i][j + (numberOfAttributesConsidered * z)]
                    z = z + 1
                trainingTargetSetReshapedAndAveraged[i][j] = trainingTargetSetReshapedAndAveraged[i][j] / numberOfElectrodesLinesToBeAveraged
            else:
                trainingTargetSetReshapedAndAveraged[i][j] = trainingTargetSetReshaped[i][j]


    for i in range(trainingTargetSetReshaped.shape[0]):
        for j in range(trainingTargetSetReshaped.shape[1]):
            if j < timeDomainNumberOfDataFlattened and j >= totalNumberOfAttributesToJump and j < trainingNotTargetSetReshaped.shape[1] - totalNumberOfAttributesToJump:
                z = 0
                while z <= numberOfElectrodesLinesToBeAveragedPerSide:
                    if z != 0:
                        trainingNotTargetSetReshapedAndAveraged[i][j] += trainingNotTargetSetReshaped[i][j - (numberOfAttributesConsidered * z)]

                    trainingNotTargetSetReshapedAndAveraged[i][j] += trainingNotTargetSetReshaped[i][j + (numberOfAttributesConsidered * z)]
                    z = z + 1
                trainingNotTargetSetReshapedAndAveraged[i][j] = trainingNotTargetSetReshapedAndAveraged[i][j] / numberOfElectrodesLinesToBeAveraged
            else:
                trainingNotTargetSetReshapedAndAveraged[i][j] = trainingNotTargetSetReshaped[i][j]
                
    #END AVERAGE SMOOTHING

    # after averaging the variables are renamed
    trainingTargetSetReshaped = trainingTargetSetReshapedAndAveraged
    trainingNotTargetSetReshaped = trainingNotTargetSetReshapedAndAveraged

    # the validation data considered is from 700 until the end
    # for example if the amount of data rows we have is 804 the validations set will consist
    # of 804 - 700 = 104 for targets and 104 for not targets
    
    validationSeparatorIndex = 700
    validationTargetSetReshaped = trainingTargetSetReshaped[validationSeparatorIndex:-1]
    validationNotTargetSetReshaped = trainingNotTargetSetReshaped[validationSeparatorIndex:-1]

    #training data is formed by 700 data for targets + 700 for not targets = 1400
    trainingTargetSetReshaped = trainingTargetSetReshaped[0:validationSeparatorIndex]
    trainingNotTargetSetReshaped = trainingNotTargetSetReshaped[0:validationSeparatorIndex]

    # a variable useful to state that there is equiprobability among target and not target
    numberOfNotTarget = numberOfTargetValues

    #concatenating target and not target in a unique set
    trainingSet = np.concatenate((trainingTargetSetReshaped,trainingNotTargetSetReshaped))
    
    #creating and concatenating labels ones for target and zeros for not target
    trainingSetLabelsOnes = np.ones((validationSeparatorIndex,), dtype=int)
    trainingSetLabelsZeros = np.zeros((validationSeparatorIndex,), dtype=int)
    trainingLabels = np.concatenate((trainingSetLabelsOnes,trainingSetLabelsZeros))

    # initialize classifiers scores
    rf450Score = 0
    gradBoostScore = 0    

    #scaling data with mean 0 and STD 1
    scaler = StandardScaler()
    scaler.fit(trainingSet)
    #reassigning the data scaled to the trainingSet variable
    trainingSet = scaler.transform(trainingSet)

    #if apply PCA is true Principal component analysis is applied to the dataset
    applyPCA = False

    if applyPCA:
        #here fitting PCA
        pca = PCA(.999).fit(trainingSet)
        #transforming in lower dimensional space
        trainingSet = pca.transform(trainingSet)

    #number of folds for cross validation
    numberOfFolds = 10

    #for all the folds
    for i in range(numberOfFolds):
        #split the data in train, test, labels of training, and labels for test
        train, test, labels, testLabels = train_test_split(trainingSet,trainingLabels,test_size=0.2,random_state=42)

        # apply the data to the classifiers and sum up all the scores for later average
        rf450 = RandomForestClassifier(n_estimators=450,n_jobs=-1)
        rf450.fit(train, labels)
        rf450Score += rf450.score(test,testLabels)
        
        gradBoost = GradientBoostingClassifier()
        gradBoost.fit(train,labels)
        gradBoostScore += gradBoost.score(test,testLabels)
    
    # print the average of the scores of the classifiers in terms of accuracy
    print("grad boost score:",end="")
    print(gradBoostScore/numberOfFolds)
    print("grad boost conf:")
    print(confusion_matrix(testLabels, gradBoost.predict(test)))
    print("")
    
    print("RF-450:",end="")
    print(rf450Score/numberOfFolds)
    print("")
    
    # concatenate the validation target and not target in a unique set
    validationSet = np.concatenate((validationTargetSetReshaped,validationNotTargetSetReshaped))

    # apply the scaling to the validation data
    validationSet = scaler.transform(validationSet)

    #apply also PCA if necessary
    if applyPCA:
        validationSet = pca.transform(validationSet)

    #create the validation labels
    labelsNumber = int(validationSet.shape[0]/2)
    validationSetLabelsOnes = np.ones((labelsNumber,), dtype=int)
    validationSetLabelsZeros = np.zeros((labelsNumber,), dtype=int)
    validationLabels = np.concatenate((validationSetLabelsOnes,validationSetLabelsZeros))

    #print the validation results
    print("Grad boost validation score:",end="")
    print(gradBoost.score(validationSet,validationLabels))
    print("grad boost validation conf:")
    print(confusion_matrix(validationLabels, gradBoost.predict(validationSet)))
    print("")
    
    print("RF-450 validation score:",end="")
    print(rf450.score(validationSet,validationLabels))
    print("RF conf:")
    print(confusion_matrix(validationLabels, rf450.predict(validationSet)))
    print("")